In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib
matplotlib.use("Agg")
import scipy.io as sio

# from other_utils import np_utils, sk_utils
import numpy as np
import tensorflow as tf
import os
from termcolor import colored
import matplotlib.pyplot as plt
import matplotlib.image as mimg
import matplotlib.gridspec as gridspec
 

import model_componets as comps
from data_loader import DataLoader
from commons import transform_util as tr_util
from model_componets import *
import vis_image as vis

import random


ImportError: No module named model_componets

In [ ]:

# data_loader = DataLoader()

# train_batch_size, test_batch_size = 2, 2
lr_disc, lr_encoder, lr_decoder = 0.000002, 0.000002, 0.000002
disp_step_save, disp_step_valid = 1000, 500

# num_batches_train = data_loader.get_num_batches('train', 2 * train_batch_size)
# num_batches_test = data_loader.get_num_batches('test', 2 * test_batch_size)

os.environ['CUDA_VISIBLE_DEVICES'] = "1"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.InteractiveSession(config=config)

def get_network_params(scope):
    return tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=scope)



In [ ]:
input_pose = tf.placeholder(tf.float32, shape = [None,17,3], name='input_x')

x_real = input_pose #root relative 

x_view_norm_real, x_local_real = comps.root_relative_to_local(x_real)

encoder_real = EncoderNet(x_local_real) #local

z_real = encoder_real['z_joints']

decoder_real = DecoderNet(z_real)

x_local_recon = decoder_real['full_body_x']#local

x_recon = comps.local_to_root_relative(x_local_recon)#root relative


loss = tf.reduce_mean(tf.abs(x_real - x_recon))

param_encoder = get_network_params(scope='Encoder_net')
param_decoder = get_network_params(scope='Decoder_net')


tf.train.Saver(param_encoder).restore(sess,'./pretrained_weights/encoder_iter-1475001')
tf.train.Saver(param_decoder).restore(sess,'./pretrained_weights/decoder_iter-1475001')

# tf.train.Saver(param_encoder).restore(sess,'../pose_embedding_training_non_norm_range_one_vneck/pretrained_weights/encoder_iter-1230001')
# tf.train.Saver(param_decoder).restore(sess,'../pose_embedding_training_non_norm_range_one_vneck/pretrained_weights/decoder_iter-1230001')

In [ ]:

# data_loader.shuffle_data('train')
# batch_idx = random.choice(np.arange(train_batch_size))
# x_inputs = data_loader.get_train_data_batch(train_batch_size, batch_idx)[:90]
# print(x_inputs.shape)
# feed_dict  = {input_pose : x_inputs}
# pose_loss,predictions,pose_recon = sess.run([loss,z_real,x_recon] , feed_dict = feed_dict)
# print (predictions.shape)

k=sio.loadmat('/data/vcl/anirudh_rule_based/codes_2020/data/vneck/split/corrected_human36_train_data_center_crops_17j_vneck.mat')


g = ['_'.join(i.split('/')[-2].split(' ')[0].split('_')[:2]) for i in k['images_path']]
from collections import Counter
c = dict(Counter(g))
video_seq1 = str(np.random.choice(c.keys()))
video_seq2 = str(np.random.choice(c.keys()))
video_seq3 = str(np.random.choice(c.keys()))
video_seq4 = str(np.random.choice(c.keys()))

print(video_seq1)
print(video_seq2)
print(video_seq3)
print(video_seq4)

poses = k['poses_3d']

def gen_image_seq(video_seq):
    m = [(c,i) for c,i in enumerate(k['images_path']) if video_seq in i]
    l = [i for i in m if "58860488_video_0_90_0" in i[-1]]
    n = [i for i in m if "58860488_video_90_180_2" in i[-1]]
    o = [i for i in m if "58860488_video_180_270_4" in i[-1]]
    p = [i for i in m if "58860488_video_270_360_6" in i[-1]]
    q = [i for i in m if "58860488_video_360_450_8" in i[-1]]
    r = [i for i in m if "58860488_video_450_540_10" in i[-1]]

    def getint(name):
        name = name[1]
        basename = name.split('_')[-1]
        num = basename.split('.')[0]
        return int(num)

    l.sort(key=getint)
    l_sor = list(np.array(l).copy())

    n.sort(key=getint)
    n_sor = list(np.array(n).copy())

    o.sort(key=getint)
    o_sor = list(np.array(o).copy())

    p.sort(key=getint)
    p_sor = list(np.array(p).copy())


    q.sort(key=getint)
    q_sor = list(np.array(q).copy())

    r.sort(key=getint)
    r_sor = list(np.array(r).copy())

    lnop=np.concatenate([l_sor,n_sor,o_sor,p_sor,q_sor,r_sor])
    return lnop

v = np.random.choice(np.arange(540-120),1)[0]
image_seq1 =  gen_image_seq(video_seq1)[v:v+120][::12]
image_seq2 =  gen_image_seq(video_seq2)[v:v+120][::12]
image_seq3 =  gen_image_seq(video_seq3)[v:v+120][::12]
image_seq4 =  gen_image_seq(video_seq4)[v:v+120][::12]

# # lnop = lnop[::8]
# # x_inputs=[poses[int(lnop[0*len(lnop)][0])],
# #           poses[int(lnop[int(0.2*len(lnop))][0])],
# #           poses[int(lnop[int(0.4*len(lnop))][0])],
# #           poses[int(lnop[int(0.6*len(lnop))][0])],
# #           poses[int(lnop[int(0.8*len(lnop))][0])],
# #           poses[int(lnop[-1][0])]]

# # x_inputs= [poses[int(l_sor[0][0])],poses[int(n_sor[-1][0])]]
# # x_inputs = [poses[int(i[0])] for i in lnop]

# used earlier
'''
x_inputs = [poses[int(i[0])] for i in image_seq1] + [poses[int(i[0])] for i in image_seq2] + [poses[int(i[0])] for i in image_seq3] + [poses[int(i[0])] for i in image_seq4]
feed_dict  = {input_pose : x_inputs}
pose_loss,predictions,pose_recon = sess.run([loss,z_real,x_recon] , feed_dict = feed_dict)
'''

x_inputs = sio.loadmat('./pose_grid_4/corner_plots/plot_4/corner_artifacts.mat')['corner_poses']




In [ ]:
%matplotlib inline 

fig = vis.get_figure()
idx=0
ax = vis.get_ax(x_inputs[idx], fig)

vis.plot_skeleton_and_scatter(x_inputs[idx], ax)

'''
0 --> top left 
1 --> top right 
2 --> bottom left
3 --> bottom right
'''

joint_names = [
    'pelvis', 'neck',  # 2
    'right_shoulder', 'right_elbow', 'right_wrist',  # 5
    'left_shoulder', 'left_elbow', 'left_wrist',  # 8
    'head_top',  # 9
    'right_hip', 'right_knee', 'right_ankle', 'right_foot',  # 13
    'left_hip', 'left_knee', 'left_ankle', 'left_foot'  # 17
]


In [ ]:
def pose_rotate(points, theta, batch_size):
    theta = theta * np.pi / 180.0
    cos_vals = np.cos(theta)
    sin_vals = np.sin(theta)
    row_1 = np.concatenate([cos_vals, -sin_vals], axis=1)# 90 x 2
    row_2 = np.concatenate([sin_vals, cos_vals], axis=1)# 90 x 2
    row_12 = np.stack((row_1, row_2), axis=1)#90 x 2 x 2
    zero_size_row1x2 = np.zeros([batch_size, 1, 2])#90 x 1 x 2
    r1x2xZero = np.concatenate([row_12, zero_size_row1x2], axis=1)
    stacker = np.array([0.0, 0.0, 1.0])
    third_cols = np.reshape(np.tile(stacker, batch_size), [batch_size, 3])
    third_cols = np.expand_dims(third_cols, 2)
    rotation_matrix = np.concatenate([r1x2xZero, third_cols], axis=2)
    return np.matmul(points.reshape([points.shape[0], 17, 3]), rotation_matrix)


def rotate_y_axis(points,theta,batch_size):
    theta = theta * np.pi / 180
    cos_vals = np.cos(theta)#90 x 1
    sin_vals = np.sin(theta)
    zero_vals = np.zeros((batch_size,1))
    ones_vals = np.ones((batch_size,1))
    row_1 = np.concatenate([cos_vals, zero_vals],axis =1)#90 x2
    row_2 = np.concatenate([zero_vals ,ones_vals],axis=1)# 90 x 2
    row_12 = np.stack((row_1, row_2), axis=1)#90 x 2 x 2
    temp_3 = np.stack((-sin_vals,zero_vals),axis =2)#90 x 1 x 2
    temp_32 = np.concatenate([row_12,temp_3],axis = 1)#90 x 3 x 2
    third_cols = np.concatenate([sin_vals,zero_vals,cos_vals],axis=1)#90 x 3
    third_cols = np.expand_dims(third_cols, 2)
    rotation_matrix = np.concatenate([temp_32, third_cols], axis=2)
    return np.matmul(points.reshape([points.shape[0], 17, 3]), rotation_matrix)


def augment_pose_seq(pose_seq,z_limit=(0,360),y_limit=(-90,90)):
    pose_seq = np.expand_dims(pose_seq, axis=1)
    thetas = np.random.uniform(z_limit[0],z_limit[1], pose_seq.shape[0])
    thetas = np.stack([thetas]*pose_seq.shape[1], 1)
    k=[]
    for ct, xx in enumerate(thetas):
        k.append(pose_rotate(pose_seq[ct], np.expand_dims(thetas[ct], 1), pose_seq[ct].shape[0]))
    k = np.stack(k, 0)

    thetas = np.random.uniform(y_limit[0],y_limit[1], k.shape[0])
    thetas = np.stack([thetas]*k.shape[1], 1)
    p=[]
    for ct, xx in enumerate(thetas):
        p.append(rotate_y_axis(k[ct], np.expand_dims(thetas[ct], 1), k[ct].shape[0]))
    p = np.stack(p, 0)
    return k


def x_flip(sk_17,neg):
    left_indices = [2, 3, 4, 9, 10, 11, 12]
    right_indices = [5, 6, 7, 13, 14, 15, 16]
    sk_17_flip = sk_17.copy()
    l_idx, r_idx = left_indices, right_indices
    sk_17_flip[:, l_idx] = sk_17[:, r_idx]
    sk_17_flip[:, r_idx] = sk_17[:, l_idx]
    sk_17_flip[:, :, 0] *= neg
    sk_17 = sk_17_flip
    return sk_17

x_inputs_flip = x_flip(x_inputs,-1)
x_inputs_flip = np.squeeze(augment_pose_seq(x_inputs_flip , z_limit=(180,180),y_limit=(0,0)))
# x_inputs_flip = x_flip(x_inputs_flip,-1)

from commons import skeleton_utils

x_inputs_flip = [skeleton_utils.fit_skeleton_frame(i) for i in x_inputs_flip ]

fig = vis.get_figure()
ax = vis.get_ax(x_inputs_flip[idx], fig )

vis.plot_skeleton_and_scatter(x_inputs_flip[idx], ax)


In [ ]:
x_inputs = [x_inputs[0] , x_inputs_flip[0] , x_inputs[2] , x_inputs[3]]
feed_dict  = {input_pose : x_inputs}
pose_loss,predictions,pose_recon = sess.run([loss,z_real,x_recon] , feed_dict = feed_dict)

In [ ]:
# mean = np.mean(predictions, axis=0)
# std = np.std(predictions,axis=0)

# print mean.shape
# print std.shape

# # x_pred = dms.decode(predictions)
# # x_pred.shape

# # imgs = sk_utils.get_skeleton_images(x_pred[0], title_prefix='p')
# # len(imgs)
# print('minimum vaue of mean ', min(mean))
# print('minimum value of std ', min(std))
# print('maximum value of mean', max(mean))
# print('maximum value of std', max(std))
sample_1 = np.reshape(predictions[0] , (1,32)) # top left 

sample_2 = np.reshape(predictions[1] , (1,32)) # top right 

sample_3 = np.reshape(predictions[2] , (1,32)) # bottom left 

sample_4 = np.reshape(predictions[3] , (1,32)) # bottom right 

In [ ]:
# sample_1 = mean + (4*std)
# sample_2 = mean -(4*std)
# sample_1 = np.reshape(sample_1 , (1,32))
# sample_2 = np.reshape(sample_2 , (1,32))
# map_d = 10


In [ ]:
'''
sample_1 = predictions[:10][np.random.choice(10)]
sample_1 = np.reshape(sample_1 , (1,32))

sample_2 = predictions[10:20][np.random.choice(10)]
sample_2 = np.reshape(sample_2 , (1,32))

sample_3 = predictions[20:30][np.random.choice(10)]
sample_3 = np.reshape(sample_3 , (1,32))

sample_4 = predictions[30:40][np.random.choice(10)]
sample_4 = np.reshape(sample_4 , (1,32))

'''

# sample_5 = predictions[4]
# sample_5 = np.reshape(sample_5 , (1,32))

# sample_6 = predictions[5]
# sample_6 = np.reshape(sample_6 , (1,32))


map_d = 10

In [ ]:
# -sample1 to +sample1 to -sample2 to +sample2
# 3+3+3+3

sub_map_d = 10
a = np.linspace(0,1, sub_map_d)
a = np.reshape(a, (sub_map_d,1))
vertical1 = a* (sample_2) + (1-a)*sample_1

b = np.linspace(0,1, sub_map_d)
b = np.reshape(b, (sub_map_d,1))
vertical2 = b* (sample_4) + (1-b)*(sample_3)

c = np.linspace(0,1, sub_map_d)
c = np.reshape(c, (sub_map_d,1,1))
grid= c* (vertical2) + (1-c)*(vertical1)

# print (vertical1.shape,vertical1.shape)

# vertical = np.concatenate([vertical1,vertical2],axis = 0)


# print vertical.shape



# b = np.linspace(0,1, sub_map_d)
# a = np.reshape(a, (sub_map_d,1))



# a.shape
# b = np.reshape(a,(map_d,1,1))

In [ ]:
# sub_map_d = 10
# a = np.linspace(0,1, sub_map_d)
# a = np.reshape(a, (sub_map_d,1))
# horizontal1 = a* (sample_4) + (1-a)*sample_5

# b = np.linspace(0,1, sub_map_d)
# b = np.reshape(b, (sub_map_d,1))
# horizontal2 = b* (sample_5) + (1-b)*(sample_6)



# print (horizontal1.shape,horizontal2.shape)

# horizontal = np.concatenate([horizontal1,horizontal2],axis = 0)


# print horizontal.shape


# d = np.linspace(0,1, map_d)
# e = np.reshape(d,(map_d,1,1))
# grid = e*predictions[:10] + (1-e)*predictions[10:]

In [ ]:
# vertical = a*sample_1[:,:16] + (1-a)*sample_2[:,:16]

# assert (vertical[0] == sample_2[0,:16]).all()
# vertical = a*sample_1 + (1-a)*sample_2
# vertical1 = a*sample_3 + (1-a)*sample_4




In [ ]:
grid.shape

In [ ]:
# horizontal = a*sample_1[:,16:] + (1-a)*sample_2[:,16:]

# assert (horizontal[0] == sample_2[0,16:]).all()

# print horizontal.shape

In [ ]:
# grid = np.empty((map_d,map_d,32))

In [ ]:
# for i in range(map_d):
#     for j in range(map_d):
#         grid[i,j] = np.concatenate((vertical[i], horizontal[j]))
# grid = vertical2

In [ ]:
grid.shape

In [ ]:
grid_flat = np.reshape(grid, (-1,32))

#print grid.shape
print grid_flat.shape

In [ ]:

#sess = tf.InteractiveSession(config=config)

state_ph = tf.placeholder(tf.float32 , shape = [None ,32] ,name = "pose_embedding")

def decode_pose(state_ph) :
    decoder_real = DecoderNet(state_ph)

    x_local_recon = decoder_real['full_body_x']#local

    x_recon = comps.local_to_root_relative(x_local_recon)#root relative
    
    return x_recon

pose_preds = decode_pose(state_ph)

param_decoder = get_network_params(scope='Decoder_net')
tf.train.Saver(param_decoder).restore(sess,'./pretrained_weights/decoder_iter-1475001')
# tf.train.Saver(param_decoder).restore(sess,'../pose_embedding_training_non_norm_range_one_vneck/pretrained_weights/decoder_iter-1230001')

feed_dict = {state_ph :grid_flat}
pred_ops = sess.run(pose_preds , feed_dict=feed_dict )

In [ ]:
# predictions = np.mean(predictions, axis=-1)
# predictions = np.mean(predictions, axis=-1)

# print predictions.shape

# square_shaped = pred_ops.reshape((10,10,15,3))
# print square_shaped.shape

In [ ]:
# pred_resh = pred_ops.reshape((10,10,17,3))


In [ ]:
# import glob
# data = glob.glob('./pose_grid_4/grid_plots/*/artifacts.mat')
# for i in data :
#     grid = sio.loadmat(i)['grid']
# #     grid = grid.reshape((10,10,17,3))
# #     final_poses = [grid[0][0],grid[0][9],grid[9][0],grid[9][9]]
#     image_path = i.replace('artifacts.mat','grid')
#     print (image_path)
#     get_all_plots(grid,image_path,az = 0)



In [ ]:
# %matplotlib inline 
# import vis_image as vis
# fig = vis.get_figure()
# ax = vis.get_ax(grid[9][9],fig,az = 0)
# vis.plot_skeleton_and_scatter(grid[9][9],ax)

In [ ]:
limb_parents = [0, 0, 1, 2, 3, 1, 5, 6, 1, 0, 9, 10, 11, 0, 13, 14, 15]


def get_ax(joints_3d, fig, az=0, ele=10, subplot='111'):
    x, y, z = subplot
    ax = fig.add_subplot(subplot, projection='3d')

    lim = np.max(np.abs(joints_3d))
    ax.view_init(azim=az, elev=ele)

    ax.set_xlim(-lim, lim)
    ax.set_ylim(-lim, lim)
    ax.set_zlim(-lim, lim)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')

    return ax

def plot_skeleton(ax, ske,color_ours,az, ground_size=0.5):
        lcolor = color_ours[0]
        rcolor = color_ours[1]
        ax.grid(False)
#         # Hide axes ticks
#         ax.set_xticks([])
#         ax.set_yticks([])
#         ax.set_zticks([])
#         plt.axis('off')
#         ax.set_aspect('equal')
        ax.elev = 10
        ax.azim = az
#         ax.azim = 0 
#         ax.dist = 9
#         ax.set_xlim(-10, 10)
#         ax.set_ylim(-10, 10)
#         ax.set_zlim(-10, 10)
        
        
# 
        lim = np.max(np.abs(ske))
#         ax.view_init(azim=az, elev=ele)

        ax.set_xlim(-lim, lim)
        ax.set_ylim(-lim, lim)
        ax.set_zlim(-lim, lim)
        
        ax.set_xlabel('x',labelpad=-8)
        ax.set_ylabel('y',labelpad=-8)
        ax.set_zlabel('z',labelpad = -10)
        ax.set_yticklabels([])
        ax.set_xticklabels([])
        ax.set_zticklabels([])
        ax.axis('off')       




        # ax.set_aspect('equal')

#         ax.view_init(azim=90, elev=10)

        C = [{0: lcolor,  1: rcolor}[flag] for flag in lr_flags]
        P = limb_parents

        # Make connection matrix
#         joint_lines = []
        for i in range(ske.shape[0]):
            x = [ske[i, 0], ske[P[i], 0]]
            y = [ske[i, 1], ske[P[i], 1]]
            z = [ske[i, 2], ske[P[i], 2]]
#             x = [ske[i, 0]*1.2, ske[P[i]*1.2, 0]]
#             y = [ske[i, 1]*1.2, ske[P[i]*1.2, 1]]
#             z = [ske[i, 2]*1.2, ske[P[i]*1.2, 2]]
            
            # print x,y,z
            ax.plot(x, y, z, lw=4, c=C[i])



# color_ours = ['blue',  'green']
# color_ours = ['#5879ff',  '#cc2520']

lr_flags = [
    1, 1,
    1, 1, 1,
    0, 0, 0,
    1,
    1, 1, 1, 1,
    0, 0, 0, 0,
]




def get_all_plots_images(skeleton_strips):
    m = 15
    fig=plt.figure(figsize=(1.77 * 4 * 45 , 4))
    gs = gridspec.GridSpec(1, m)
    gs.update(wspace=0.05, hspace=0.4, left = None, right = 0.6, bottom = 0.1, top = 0.9) 

    for j in range(m):
        idx = j
        ax = plt.subplot(gs[idx])
        k=mpimg.imread(skeleton_strips[j])
        ax.imshow(k)
        ax.axis('off')    

    return fig  


def get_all_plots(skeleton_strips_male,az):
    
#     l = len(os.listdir('./pose_grid_4/grid_plots/')) + 1
#     os.makedirs('./pose_grid_4/grid_plots/plot_{}/'.format(str(l)))
    path = './pose_grid_4/grid_plots/plot_flipped/'
    if not os.path.exists(path):
        os.makedirs(path)

    sio.savemat('./pose_grid_4/grid_plots/plot_flipped/artifacts.mat',{'gt':x_inputs,'embedding':grid_flat,'grid':pred_ops})

#     fig=plt.figure(figsize=(4  * 15 , 4 * 15))#width , height
    fig=plt.figure(figsize=(4  * 1 , 5))#width , height
    gs = gridspec.GridSpec(1, 1) #rows and columns
#     gs.update(wspace=0.05, hspace=0.0005) # set the spacing between axes. 
    gs.update(wspace=0.05, hspace=0.4, left = None, right = 0.6, bottom = 0.1, top = 0.9) 
    cnt = 0
    for i in range (10):
    #     lcolor, rcolor = colors
        for j in range(10):
            idx = j
#             ax = plt.subplot(gs[idx], projection='3d')
#     #         color_ours = ['blue',  'green']
#             color_ours = ['k','m']

#             plot_skeleton(ax, skeleton_strips_male[j],color_ours,az)
    #         color_ours = ['#5879ff',  '#cc2520']
    #         color_ours = ['k','m']
    #         plot_skeleton(ax, skeleton_strips_female[j],color_ours)
            ax = plt.subplot(gs[0], projection='3d')
            color_ours = ['k','m']
            plot_skeleton(ax, skeleton_strips_male[cnt],color_ours,az)
            cnt = cnt + 1
#             plt.savefig("./pose_grid_4/grid_plots/plot_{}/grid{}.png".format(str(l),i),bbox_inches = 'tight',dpi=350, transparent = True);
            plt.savefig("./pose_grid_4/grid_plots/plot_flipped/{}_{}.png".format(i,j),bbox_inches = 'tight',dpi=350, transparent = True);
#         plt.savefig("./pose_grid_4/grid_plots/plot_flipped/grid{}.png".format(i),bbox_inches = 'tight',dpi=350, transparent = True);

        
    return fig


def get_all_plots_together(skeleton_strips_male,az):
    
#     l = len(os.listdir('./pose_grid_4/grid_plots/')) + 1
#     os.makedirs('./pose_grid_4/grid_plots/plot_{}/'.format(str(l)))
#     os.makedirs('./pose_grid_4/grid_plots/plot_flipped/')

#     sio.savemat('./pose_grid_4/grid_plots/plot_{}/artifacts.mat'.format(str(l)),{'gt':x_inputs,'embedding':grid_flat,'grid':pred_ops})

#     fig=plt.figure(figsize=(4  * 15 , 4 * 15))#width , height
    fig=plt.figure(figsize=(4  * 10 , 5 * 10))#width , height
    gs = gridspec.GridSpec(10, 10) #rows and columns
#     gs.update(wspace=0.05, hspace=0.0005) # set the spacing between axes. 
    gs.update(wspace=0.05, hspace=0.4, left = None, right = 0.6, bottom = 0.1, top = 0.9) 
    cnt = 0
    for i in range (100):
    #     lcolor, rcolor = colors
        idx = i
#             ax = plt.subplot(gs[idx], projection='3d')
#     #         color_ours = ['blue',  'green']
#             color_ours = ['k','m']

#             plot_skeleton(ax, skeleton_strips_male[j],color_ours,az)
#         color_ours = ['#5879ff',  '#cc2520']
#         color_ours = ['k','m']
#         plot_skeleton(ax, skeleton_strips_female[j],color_ours)
        ax = plt.subplot(gs[i], projection='3d')
        color_ours = ['k','m']
        plot_skeleton(ax, skeleton_strips_male[i],color_ours,az)
        cnt = cnt + 1
#             plt.savefig("./pose_grid_4/grid_plots/plot_{}/grid{}.png".format(str(l),i),bbox_inches = 'tight',dpi=350, transparent = True);
#             plt.savefig("{}{}.png".format(path,i),bbox_inches = 'tight',dpi=350, transparent = True);
#         plt.savefig("./pose_grid_4/grid_plots/plot_flipped/grid{}.png".format(i),bbox_inches = 'tight',dpi=350, transparent = True);

        
    return fig



def get_tmp_plots(skeleton_strips_male,az):
    
    l = len(os.listdir('./pose_grid_4/corner_plots/')) + 1
    os.makedirs('./pose_grid_4/corner_plots/plot_{}/'.format(str(l)))
    sio.savemat('./pose_grid_4/corner_plots/plot_{}/corner_artifacts.mat'.format(str(l)),{'corner_poses':final_poses})

#     fig=plt.figure(figsize=(4  * 15 , 4 * 15))#width , height
    fig=plt.figure(figsize=(4  * 10 , 5))#width , height
    gs = gridspec.GridSpec(1, 10) #rows and columns
#     gs.update(wspace=0.05, hspace=0.0005) # set the spacing between axes. 
    gs.update(wspace=0.05, hspace=0.4, left = None, right = 0.6, bottom = 0.1, top = 0.9) 
    cnt = 0
    for i in range (2):
    #     lcolor, rcolor = colors
        for j in range(2):
            idx = j
#             ax = plt.subplot(gs[idx], projection='3d')
#     #         color_ours = ['blue',  'green']
#             color_ours = ['k','m']

#             plot_skeleton(ax, skeleton_strips_male[j],color_ours,az)
    #         color_ours = ['#5879ff',  '#cc2520']
    #         color_ours = ['k','m']
    #         plot_skeleton(ax, skeleton_strips_female[j],color_ours)
            ax = plt.subplot(gs[j], projection='3d')
            color_ours = ['k','m']
            plot_skeleton(ax, skeleton_strips_male[cnt],color_ours,az)
            cnt = cnt + 1
            plt.savefig("./pose_grid_4/corner_plots/plot_{}/grid{}.png".format(str(l),i),bbox_inches = 'tight',dpi=350, transparent = True);
#             plt.savefig("{}{}.png".format(path,i),bbox_inches = 'tight',dpi=350, transparent = True);
        
    return fig



In [ ]:
%matplotlib inline

In [ ]:
# import os 
# if not os.path.exists('./pose_grid_4/corner_plots/'):
#     os.makedirs('./pose_grid_4/corner_plots')
    
# if not os.path.exists('./pose_grid_4/grid_plots/'):
#     os.makedirs('./pose_grid_4/grid_plots/')
# # for i in range(100):
#     #pred_ops = tr_util.local_to_view_norm_skeleton_batch(pred_ops)
fig = get_all_plots(pred_ops,az = 0)
# fig = get_all_plots_together(pred_ops,az = 0)


# fig = get_tmp_plots(final_poses,az = 0)
# #plt.savefig("./pose_grid/"+str(i)+".png",bbox_inches = 'tight',dpi=350);

# print pred_ops.shape

In [ ]:
# import matplotlib.pyplot as plt
# %matplotlib inline  

In [ ]:
# plt.imshow(speed, cmap='hot', interpolation='nearest')
# plt.savefig("test.svg", format="svg")
# plt.show()

In [ ]:
# a = np.argmax(speed)

# c  = np.unravel_index(a, speed.shape)

# print c

In [ ]:
# for i in range(-4,5):
#     l = mean + i*std
#     l = np.reshape(l, (1,128))
#     x_pred = sess.run(pred_ops_ph,feed_dict={state_ph : l})
#     imgs = sk_utils.get_skeleton_images(x_pred[0], title_prefix='p')
#     writer = imageio.get_writer('./predictions/op_{}.mp4'.format(i), fps=30)
#     for j, im in enumerate(imgs):
#         writer.append_data(im)
#         if j % 10 == 0:
#             print 'done with %d frames' % j
#     writer.close()



In [ ]:
# # !rm -rf pose_grid/plot_4/
# !ls pose_grid/



In [ ]:
# %matplotlib inline

In [ ]:
# fig = plt.figure()
# ax = vis.get_ax(poses[int(l_sor[0][0])], fig, az=0)
# vis.plot_skeleton_and_scatter(poses[int(l_sor[0][0])], ax)

In [ ]:
# fig = plt.figure()
# ax = vis.get_ax(poses[int(n_sor[-1][0])], fig, az=0)
# vis.plot_skeleton_and_scatter(poses[int(n_sor[-1][0])], ax)

In [ ]:
# m = [(c,i) for c,i in enumerate(k['images_path']) if "" in i]


In [ ]:
# print (m[:10])


In [ ]:
# m = [(c,i) for c,i in enumerate(k['images_path']) if "S7_Eating" in i]
# l = [i for i in m if "58860488_video_0_90_0" in i[-1]]
# n = [i for i in m if "58860488_video_90_180_2" in i[-1]]
# o = [i for i in m if "58860488_video_180_270_4" in i[-1]]
# p = [i for i in m if "58860488_video_270_360_6" in i[-1]]




In [ ]:
# from collections import Counter

In [ ]:
# tmp = [ '_'.join(i[1].split('/')[7].replace(' ','_').split('_')[:2]) for i in m]

In [ ]:
# Counter(tmp)